In [14]:
import numpy as np
import pandas as pd
import cvxpy as cp
import mosek
import matplotlib.pyplot as plt
import affine_approx as af
import importlib
import scipy.stats
from scipy.stats import norm, chi2, wishart, beta, lognorm

In [6]:
def kl_ca_conj(x):
    return(1-np.exp(-x))

def rk_eval(X,delta, phi_conj):
    n = len(X)
    som = 0
    X = np.sort(X)
    X_diff = np.concatenate((np.array([X[0]]),np.diff(X)))
    for i in range(n):
        som = som + phi_conj(delta*(n-i)/n)*X_diff[i]
    return(som/delta)

def sd(X):
    n = len(X)
    som = 0
    m = np.mean(X)
    for i in range(n):
        som = som + (X[i]-m)**2
    return(np.sqrt(som/n))

def miniance(X):
    n = len(X)
    som = 0
    X = np.sort(X)
    X_diff = np.concatenate((np.array([X[0]]),np.diff(X)))
    for i in range(n):
        som = som + ((n-i)/n)**2*X_diff[i]
    return(som)

def h_ddro(x,delta):
    teller = 1-np.exp(-delta*x)
    noemer = 1-np.exp(-delta)
    return(teller/noemer)


def argmax_kl_dtconj(x2,x1,delta):
    frac = (np.exp(-delta*x1)-np.exp(-delta*x2))/(x2-x1)
    return(-np.log(frac/delta)/delta)

def max_kl_dtconj(x2,x1,x,delta):
    frac = (h_ddro(x2,delta)-h_ddro(x1,delta))/(x2-x1)
    return(h_ddro(x,delta)-frac*(x-x1)-h_ddro(x1,delta))

def emp_likelihood_lb(X, r):
    n = len(X)
    theta = cp.Variable()
    lbda = cp.Variable(nonneg=True)
    s = cp.Variable(n, nonneg=True)
    constraints = [s == lbda - theta + X]
    f_obj = -cp.sum(cp.kl_div(lbda, s) + lbda - s)
    obj = cp.Maximize(f_obj / n + theta - lbda * r)
    prob = cp.Problem(obj, constraints)
    prob.solve()

    return prob.value


def emp_likelihood_ub(X, r):
    n = len(X)
    theta = cp.Variable()
    lbda = cp.Variable(nonneg=True)
    s = cp.Variable(n, nonneg=True)
    constraints = [s >= 0, s == lbda + theta - X]
    f_obj = cp.sum(cp.kl_div(lbda, s) + lbda - s)
    obj = cp.Minimize(f_obj / n + theta + lbda * r)
    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.MOSEK)

    return prob.value


def entropy_lb(X,r):
    n = len(X)
    theta = cp.Variable(1)
    lbda = cp.Variable(1, nonneg = True)
    t = cp.Variable(n)
    constraints = [theta-X <= -cp.kl_div(lbda,t)-lbda + t]
    #for i in range(n):
    #    constraints.append(theta - X[i] <= -cp.kl_div(lbda,t[i])-lbda + t[i])
    obj = cp.Maximize((1-r)*lbda + theta -cp.sum(t)/n)
    prob = cp.Problem(obj, constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value)

def entropy_ub(X,r):
    n = len(X)
    theta = cp.Variable(1)
    lbda = cp.Variable(1, nonneg = True)
    t = cp.Variable(n)
    constraints = [X-theta <= -cp.kl_div(lbda,t)-lbda + t]
    #for i in range(n):
    #    constraints.append(X[i]-theta <= -cp.kl_div(lbda,t[i])-lbda + t[i])
    obj = cp.Minimize((r-1)*lbda + theta +cp.sum(t)/n)
    prob = cp.Problem(obj, constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value)
    
    
def portfolio_ddro(R, slope, const):
    n, I = R.shape
    K = len(slope)
    a = cp.Variable(I, nonneg=True)
    beta = cp.Variable()
    lbda = cp.Variable((n, K), nonneg=True)
    v = cp.Variable(K, nonneg=True)
    p = np.ones(n) / n
    c = cp.Variable()
    expected_returns = R @ a
    constraints = [
        cp.sum(a) == 1,
        cp.abs(a) <= 1,
        expected_returns - cp.sum(lbda, axis=1) - beta <= 0,  # Vectorized
        lbda <= cp.reshape(v, (1, K),order='F')  # Ensure v is broadcasted to shape (1, K)
    ]
    
    constraints.append(beta + v @ const + p @ (lbda @ slope) <= c)
    obj = cp.Minimize(c)
    prob = cp.Problem(obj, constraints)
    prob.solve()
    
    return a.value, prob.value
    

def portfolio_dro(R, r):
    n, I = R.shape
    a = cp.Variable(I, nonneg=True)
    eta = cp.Variable()
    lbda = cp.Variable(nonneg=True)
    t = cp.Variable(n)
    p = np.ones(n) / n
    expected_returns = R @ a
    kl_term = -cp.kl_div(lbda, t) - lbda + t
    constraints = [
        cp.sum(a) == 1,
        cp.abs(a) <= 1,
        expected_returns - eta <= kl_term
    ]
    obj = cp.Minimize((r - 1) * lbda + eta + cp.sum(t) / n)
    prob = cp.Problem(obj, constraints)
    mosek_params2 = {
        'MSK_DPAR_INTPNT_CO_TOL_REL_GAP': 1e-7,  # Relative gap tolerance
        'MSK_DPAR_INTPNT_CO_TOL_MU_RED': 1e-7,   # Complementarity gap tolerance
        'MSK_DPAR_INTPNT_CO_TOL_PFEAS': 1e-8,    # Primal feasibility tolerance
        'MSK_DPAR_INTPNT_CO_TOL_DFEAS': 1e-8     # Dual feasibility tolerance
    }
    
    prob.solve(solver=cp.MOSEK, mosek_params=mosek_params2)
    
    return a.value, prob.value


def portfolio_dro_chi2(R,r):
    n = len(R)
    I = len(R[0])
    a = cp.Variable(I)
    eta = cp.Variable(1)
    lbda = cp.Variable(1, nonneg = True)
    t = cp.Variable(n)
    w = cp.Variable(n, nonneg = True)
    p = np.zeros(n) + 1/n
    constraints = [cp.sum(a)==1, cp.abs(a)<=1]
    for i in range(n):
        constraints.append(((R @ a)[i] - eta)/2 + lbda <= w[i])
        constraints.append(cp.norm(cp.vstack([w[i],t[i]/2]))<=(t[i]+2*lbda)/2)
    obj = cp.Minimize(r*lbda+eta+cp.sum(t)/n)
    prob = cp.Problem(obj,constraints)
    prob.solve()
    return(a.value, prob.value)


def portfolio_saa(R):
    n = len(R)
    I = len(R[0])
    a = cp.Variable(I,nonneg = True)
    constraints = [cp.sum(a)==1, cp.abs(a)<=1]
    p = np.zeros(n) + 1/n
    obj = cp.Minimize(p@(R@a))
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK)
    return(a.value, prob.value)

def portfolio_true(mu):
    I = len(mu)
    a = cp.Variable(I,nonneg = True)
    constraints = [cp.sum(a)==1, cp.abs(a)<= 1]
    obj = cp.Minimize(a@mu)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver=cp.MOSEK)
    return(a.value, prob.value)
    

    

In [8]:

N_size = np.array([50, 100,200,500,1000,2000])
eps = 0.0001
a_x, b_x = 6, 8  # Range for uniform part of Z
M = 26           # Constant value for Z
epsilon = 0.05    # Probability of taking value M
mean_x = (a_x+b_x)/2 +epsilon*M
a_z, b_z = (mean_x+0.1)-8, (mean_x+0.1)+8
# Generate samples
rep = 100
saa_w = np.zeros((len(N_size),rep))
dro_w = np.zeros((len(N_size),rep))
ddro_w = np.zeros((len(N_size),rep))
np.random.seed(10)
for i in range(len(N_size)):
    for j in range(rep):
        if j % 20 == 0:
            print(j)
        X = np.random.uniform(a_x,b_x,size = N_size[i])+np.random.binomial(1,epsilon,size = N_size[i])*M
        Y = np.random.uniform(a_z,b_z,size = N_size[i]) 
        R = np.column_stack((X, Y))
        r = chi2.ppf(0.999,1)/(2*N_size[i])
        r2 = np.sqrt(2*chi2.ppf(0.999,1))
        delta = r2/np.sqrt(N_size[i])
        x_points = af.affine_approx(eps,argmax_kl_dtconj,max_kl_dtconj,delta)
        if len(x_points)==2:
            print('true')
        else:
            [slope, const] = af.makepoints(h_ddro,x_points,delta)
        saa_w[i,j] = portfolio_saa(R)[0][0]
        dro_w[i,j] =  portfolio_dro(R, r)[0][0]
        ddro_w[i,j] = portfolio_ddro(R, slope, const)[0][0]
    

0
20
40
60
80
0
20
40
60
80
0
20
40
60
80
0
20
40
60
80
0
20
40
60
80
0
20
40
60
80


In [10]:
for i in range(len(N_size)):
    print('N=', N_size[i])
    #print('SAA', np.mean(saa_w[i]))
    #print('DRO',np.mean(dro_w[i]))
    #print('DDRO',np.mean(ddro_w[i]))
    print('DRO mu1<mu2',np.mean(dro_w[i][saa_w[i]<1]))
    print('DDRO mu1<mu2',np.mean(ddro_w[i][saa_w[i]<1]))
    print('DRO mu1>mu2',np.mean(dro_w[i][saa_w[i]>0]))
    print('DDRO mu1>mu2',np.mean(ddro_w[i][saa_w[i]>0]))

N= 50
DRO mu1<mu2 0.19225810415469174
DDRO mu1<mu2 0.20603512432035878
DRO mu1>mu2 0.5444258972937279
DDRO mu1>mu2 0.9879286616241635
N= 100
DRO mu1<mu2 0.20073203458680225
DDRO mu1<mu2 0.24031043117035283
DRO mu1>mu2 0.46545885681660915
DDRO mu1>mu2 0.979342261926556
N= 200
DRO mu1<mu2 0.24168278683262323
DDRO mu1<mu2 0.34764821918427435
DRO mu1>mu2 0.5130060385155943
DDRO mu1>mu2 0.9854314197095522
N= 500
DRO mu1<mu2 0.2107691295279975
DDRO mu1<mu2 0.19821233305541083
DRO mu1>mu2 0.4793101588295459
DDRO mu1>mu2 0.9856026540144062
N= 1000
DRO mu1<mu2 0.2635497454156693
DDRO mu1<mu2 0.33859562872593807
DRO mu1>mu2 0.5643345046840412
DDRO mu1>mu2 0.9900681644350662
N= 2000
DRO mu1<mu2 0.2803494822552062
DDRO mu1<mu2 0.3282660325859105
DRO mu1>mu2 0.558041797349134
DDRO mu1>mu2 0.9915579275155081


In [12]:
print(np.mean(ddro_w))
print(np.mean(dro_w))
print(np.mean(saa_w))
print(np.mean(dro_w[saa_w<1]))
print(np.mean(ddro_w[saa_w<1]))
print(np.mean(dro_w[saa_w>0]))
print(np.mean(ddro_w[saa_w>0]))

0.7085367357942229
0.4077888150973619
0.61
0.22650073653498884
0.2729066171999812
0.5236943079487153
0.9870543526003775
